## Table of Contents
- [Import libraries](#1)
- [processing tables](#2)
- [Download tables](#3)


<a name='1'></a>
## Import libraries

The script is focused on setting up an environment for data analysis and visualization. It imports a suite of libraries and modules that are essential for statistical computing, data manipulation, progress tracking, file system operations, and generating visualizations such as plots and Venn diagrams. The specific libraries imported include pandas for data structures, numpy for numerical operations, tqdm for progress bars, glob for file path retrieval, os for operating system interaction, matplotlib and seaborn for plotting and graphical representations, and matplotlib_venn for creating Venn diagrams.

Additionally, the script modifies the system path to include a custom directory, which suggests that the script will use additional custom modules and configuration settings located in this directory. These custom modules, imported with wildcard imports (from config import * and from functions import *)

In [1]:
# %load /cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/TASKS/func/base_imports.py
import pandas as pd
import numpy as np
import tqdm 
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tarfile
import re
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
import sys
sys.path.append(r"/cluster/home/myurchikova/github/projects2020_ohsu/eth/learning_Master_thesis/TASKS/func")
from config import *
from functions import *

<a name='2'></a>
## Processing table

Establishing directories for data storage and retrieval.
Initializing an empty DataFrame to store filtered results.
Employing conditional logic to handle different versions or types of data.
Reading and loading data from various files, with conditions to filter and select specific data points.
Mapping related data files to one another for comparative analysis.
Manipulating a DataFrame by appending new data, which includes sample information, filtering criteria, and measurement sizes.
Assigning priority levels to data entries, indicating a method for ranking their importance or relevance.
Printing information for verification or logging purposes.
Aggregating and consolidating data in preparation for further analysis or visualization.

In [4]:
# Loading tables according to given paths

filter_dir = os.path.join(FILTERING_PATH_BRCA, 'filtering_samples', FILTERING_ID)

out_df_filtered=pd.DataFrame()

out_var=''
title_venn = '{sample}'

# ETH Names
# OLD code
if not OHSU_BRCA_NEW: 
    eth_all = glob.glob(os.path.join(filter_dir, 'G*'))
    salt=''
    file=TAR_OHSU_BRCA
    
#New code
if OHSU_BRCA_NEW:
    eth_all = glob.glob(os.path.join(filter_dir, 'G*'))
    eth_all = [i for i in eth_all if len(os.path.basename(i)) == 45] #Shorten path has len 45
    salt='OHSU_BRSA_NEW'
    file=TAR_OSHU_BRCA_NEW
    
# OHSU Names
with tarfile.open(file, "r:*") as tar:
    ohsu_all = tar.getnames()

print(ohsu_all[0],TT)
# Get file pairs
file_pair = {}
for idx_eth, eth in enumerate(eth_all):
    pattern = os.path.basename(eth).replace('G_', '').replace('.gz', '') 
    for idx_ohsu, ohsu in enumerate(ohsu_all):
        if pattern in ohsu:
            file_pair[eth] = ohsu
        
restricts = RESTRICTS_BRCA
for restrict in restricts:
    df = {'sample' : [], 
      'filter_foreground' : [], 
      'filter_background' : [], 
      'filter': [],
      'size_ohsu' : [], 
      'size_eth' : [], 
      'size_intersection' : [], 
      'size_ohsu\eth' : [], 
      'size_eth\ohsu' : [],
      'eth_kmers\inter':[],
      'ohsu_kmers\inter':[],
      'inter_kmers':[],
          
      'coord_OHSU':[],
      'coord_ETH':[],
      'size_ohsu_coor' : [], 
      'size_eth_coor' : [], 
      'size_intersection_coor' : [], 
      'size_ohsu\eth_coor' : [], 
      'size_eth\ohsu_coor' : [],
      'eth_coor\inter_coor':[],
      'ohsu_coor\inter_coor':[],
      'inter_coor':[],
      'eth_coor\ohsu_coor':[],
      'ohsu_coor\eth_coor':[],
      'priority':[],
         }
    with tarfile.open(file, "r:*") as tar: #OHSU
        for eth, ohsu in file_pair.items(): # ETH
            print('eth',eth,TT,'ohsu',ohsu)
            if (not restrict) or restrict == re.findall('G_([\s\S]+?)_',eth)[0].replace('-',''): #Restrict to category of interest
                    print(restrict)
                    df_ohsu = pd.read_csv(ohsu_path:=tar.extractfile(ohsu), sep="\t")
                    df_ohsu.reset_index(inplace=True)
                    if not df_ohsu.empty: df_ohsu = table_processing.ohsu_to_eth_coord(df_ohsu)
                    if not df_ohsu.empty: df_ohsu['junction_coordinate'] = df_ohsu['jx_shifted'].apply(lambda x: ':'.join(x.split(';')[1:3]))
                    df_eth = pd.read_csv(eth, sep="\t")
                    if not df_eth.empty: df_eth=table_processing.get_junction_coordinates(df_eth,'coord')
                    df1=df_eth
                    df2=df_ohsu
                    print(df_eth.columns,df_eth,sep='\n')
                    print(df_ohsu.columns,df_ohsu,sep='\n')
                    df_eth = set(df_eth['kmer'])
                    df_ohsu = set(df_ohsu['index'])
                    df_eth_coor = set(df1['junction_coordinate']) if not df1.empty else set([])
                    df_ohsu_coor = set(df2['junction_coordinate']) if not df1.empty else set([])
                    name = os.path.basename(ohsu).replace('.tsv', '').split('_')
                    print(name)
                    df['coord_ETH'].append(df1['junction_coordinate'] if not df1.empty else 'None' )
                    df['coord_OHSU'].append(df2['junction_coordinate'] if not df2.empty else 'None')
                    df['sample'].append(ss:=name[1].replace('-',''))
                
                    if not OHSU_BRCA_NEW:
                        df['filter_foreground'].append(name[2])
                        df['filter_background'].append(name[3])
                        if name[3][1] == 'Any':
                            priority=0
                        elif name[3][1] == 10:
                            priority=1
                        elif name[3][1] == 2:
                            priority=2
                        else:
                            priotity = None
                        df['priority'].append(priority)
                        df['filter'].append(name[2]+' '+name[3])
                    else:
                        df['filter'].append(name[2])
                        a = []
                        for i in range(5):
                            if name[2][i] == 'A':
                                a.append('Any')
                            elif name[2][i] == 'X':
                                a.append('10')
                            elif name[2][i] == 'N':
                                a.append('None')
                            else:
                                a.append(name[2][i])
                        df['filter_foreground'].append(f'({a[0]}, {a[1]}, {a[2]})')
                        df['filter_background'].append(f'({a[3]}, {a[4]})')
                        if a[4] == 'Any':
                            priority=0
                        elif a[4] == '10':
                            priority=1
                        elif a[4] == '2':
                            priority=2
                        else:
                            priority = None
                        df['priority'].append(priority)
                    print(a)
                    print(priority)
                    df['size_ohsu'].append(len(df_ohsu))
                    df['size_eth'].append(len(df_eth))
                    df['size_ohsu_coor'].append(len(df_ohsu_coor))
                    df['size_eth_coor'].append(len(df_eth_coor))
                    df['size_ohsu\eth'].append(len(df_ohsu_filter:=df_ohsu.difference(df_eth)))
                    df['size_eth\ohsu'].append(len(df_eth_filter:=df_eth.difference(df_ohsu)))
                
                    if name[2] in ['0AN3XGA','0AN3AGA'] and name[1].replace('-','') == 'TCGAC8A12P01A11RA11507':
                       out_var+= f'sample: {ss}\n filter_for: ({a[0]}, {a[1]}, {a[2]})\n  filter_back: ({a[3]}, {a[4]})\n ohsu/eth: {len(df_ohsu_filter)}\n path_OSHU: {ohsu_path}\n path_ETH: {eth}\n'
                    
                    df['size_ohsu\eth_coor'].append(len(df_ohsu_filter_coor:=df_ohsu_coor.difference(df_eth_coor)))
                    df['size_eth\ohsu_coor'].append(len(df_eth_filter_coor:=df_eth_coor.difference(df_ohsu_coor)))
                    df['ohsu_coor\eth_coor'].append(np.array(df_ohsu_filter_coor))
                    print(len(df_ohsu_filter_coor))
                    print(len(df_eth_filter_coor))
                    df['eth_coor\ohsu_coor'].append(np.array(df_eth_filter_coor))
                    df['size_intersection'].append(len(df_inter_filter:=set(df_ohsu) & set(df_eth)))
                    df['size_intersection_coor'].append(len(df_inter_filter_coor:=df_ohsu_coor & df_eth_coor))
                    print(len(df_inter_filter_coor))
                    print('NEXT')
                    df['eth_kmers\inter'].append(df_eth_witout_inter:=list(df_eth_filter.difference(df_inter_filter)))
                    df['ohsu_kmers\inter'].append(df_ohsu_witout_inter:=list(df_ohsu_filter.difference(df_inter_filter)))
                    df['eth_coor\inter_coor'].append(df_eth_witout_inter_coor:=list(df_eth_filter_coor.difference(df_inter_filter_coor)))
                    df['ohsu_coor\inter_coor'].append(df_ohsu_witout_inter_coor:=list(df_ohsu_filter_coor.difference(df_inter_filter_coor)))
                    df['inter_kmers'].append(list(df_inter_filter))
                    df['inter_coor'].append(list(df_inter_filter_coor))
    df = pd.DataFrame(df)
         #print(df)
    if not out_df_filtered.empty:
        out_df_filtered = pd.concat([out_df_filtered, df])
    else:
        out_df_filtered = df

J_TCGA-24-1431-01A-01R-1566-13_02101GA.tsv 
---------------------------------------------

eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-A2-A0D2-01A-21R-A034-07_02501GA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-A2-A0D2-01A-21R-A034-07_02501GA.tsv
eth /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102/filtering_samples/filters_19May_order_5ge_wAnnot_GPstar/G_TCGA-C8-A12P-01A-11R-A115-07_0A512GA.tsv.gz 
---------------------------------------------
 ohsu J_TCGA-C8-A12P-01A-11R-A115-07_0A512GA.tsv
TCGAC8A12P01A11RA11507


46it [00:00, 6384.02it/s]


Index(['kmer', 'coord', 'junctionAnnotated', 'readFrameAnnotated', 'strand',
       'junction_coordinate'],
      dtype='object')
         kmer                                              coord  \
0   GALVYAAKP      54634783:54634802:54632373:54632381:None:None   
1   QFIENRVLF      54631737:54631750:54629607:54629621:None:None   
2   AGALVYAAK      54634783:54634805:54632376:54632381:None:None   
3   YAAKPNEEI      54634783:54634790:54632361:54632381:None:None   
4   KTIHRKQSP      54631737:54631754:54629611:54629621:None:None   
5   NSSKTESCS      54631737:54631749:54629606:54629621:None:None   
6   GGKQFIENR      54631737:54631759:54629616:54629621:None:None   
7   LVYAAKPNE      54634783:54634796:54632367:54632381:None:None   
8   HRKQSPVLS      54631737:54631745:54629602:54629621:None:None   
9   VAGALVYAA      54634783:54634808:54632379:54632381:None:None   
10  GKQFIENRV      54631737:54631756:54629613:54629621:None:None   
11  FIENRVLFC      54631737:54631747:54629604:54629621

207it [00:00, 3497.51it/s]


Index(['kmer', 'coord', 'junctionAnnotated', 'readFrameAnnotated', 'strand',
       'junction_coordinate'],
      dtype='object')
          kmer                                          coord  \
0    GALVYAAKP  54634783:54634802:54632373:54632381:None:None   
1    QFIENRVLF  54631737:54631750:54629607:54629621:None:None   
2    AGALVYAAK  54634783:54634805:54632376:54632381:None:None   
3    YAAKPNEEI  54634783:54634790:54632361:54632381:None:None   
4    GDLDSVLGA  54633062:54633087:54632379:54632381:None:None   
..         ...                                            ...   
202  RGYGKINKQ  73291597:73291621:73291177:73291180:None:None   
203  GYGKINKQI  73291597:73291618:73291174:73291180:None:None   
204  NKQITSCGP  73291597:73291603:73291159:73291180:None:None   
205  KQITSCGPS  73291597:73291600:73291156:73291180:None:None   
206  YGKINKQIT  73291597:73291615:73291171:73291180:None:None   

     junctionAnnotated  readFrameAnnotated strand junction_coordinate  
0                

...

Index(['kmer', 'coord', 'junctionAnnotated', 'readFrameAnnotated', 'strand',
       'junction_coordinate'],
      dtype='object')
         kmer                                              coord  \
0   VELEAKFET      52238706:52238724:52243011:52243020:None:None   
1   VELEAKFET      52238706:52238724:52243011:52243020:None:None   
2   VELEAKFET      52238706:52238724:52243011:52243020:None:None   
3   VELEAKFET      52238706:52238724:52243011:52243020:None:None   
4   LEAKFETLQ      52238712:52238724:52243011:52243026:None:None   
..        ...                                                ...   
72  TLRWDANLR  124913290:124913297:124913073:124913093:None:None   
73  IFVKTLRWD  124913290:124913309:124913085:124913093:None:None   
74  IFVKTLRWD  124913290:124913309:124913085:124913093:None:None   
75  IFVKTLRWD  124913290:124913309:124913085:124913093:None:None   
76  IFVKTLRWD  124913290:124913309:124913085:124913093:None:None   

    junctionAnnotated  readFrameAnnotated strand  jun

180it [00:00, 2756.06it/s]


Index(['kmer', 'coord', 'junctionAnnotated', 'readFrameAnnotated', 'strand',
       'junction_coordinate'],
      dtype='object')
          kmer                                              coord  \
0    VELEAKFET      52238706:52238724:52243011:52243020:None:None   
1    VELEAKFET      52238706:52238724:52243011:52243020:None:None   
2    VELEAKFET      52238706:52238724:52243011:52243020:None:None   
3    VELEAKFET      52238706:52238724:52243011:52243020:None:None   
4    LEAKFETLQ      52238712:52238724:52243011:52243026:None:None   
..         ...                                                ...   
175  EGSPSADFA  136407428:136407432:136407204:136407227:None:None   
176  EEGSPSADF  136407428:136407435:136407207:136407227:None:None   
177  DLLDEEEGS  136407428:136407450:136407222:136407227:None:None   
178  LLDEEEGSP  136407428:136407447:136407219:136407227:None:None   
179  EEEGSPSAD  136407428:136407438:136407210:136407227:None:None   

     junctionAnnotated  readFrameAnnotate

447it [00:00, 4812.74it/s]

Index(['kmer', 'coord', 'junctionAnnotated', 'readFrameAnnotated', 'strand',
       'junction_coordinate'],
      dtype='object')
          kmer                                              coord  \
0    DKLFQSWSS      89497170:89497175:89421235:89421257:None:None   
1    DKLFQSWSS      89497170:89497175:89421235:89421257:None:None   
2    DKLFQSWSS      89497170:89497175:89421235:89421257:None:None   
3    DKLFQSWSS      89497170:89497175:89421235:89421257:None:None   
4    IPDKLFQSW      89497170:89497181:89421241:89421257:None:None   
..         ...                                                ...   
442  EGSPSADFA  136407428:136407432:136407204:136407227:None:None   
443  EEGSPSADF  136407428:136407435:136407207:136407227:None:None   
444  DLLDEEEGS  136407428:136407450:136407222:136407227:None:None   
445  LLDEEEGSP  136407428:136407447:136407219:136407227:None:None   
446  EEEGSPSAD  136407428:136407438:136407210:136407227:None:None   

     junctionAnnotated  readFrameAnnotate

<a name='3'></a>
## Download tables

It uses a custom create_path function to build file paths for data storage and retrieval, with the paths being contingent on whether a certain condition (represented by OHSU_BRCA_NEW) is true or false.
The script appears to switch between different directory paths based on this condition, suggesting that it manages multiple versions or sets of data (e.g., "new" versus "old").
Paths for different types of data (FILTERING_BRCA, ETH_TASK_BRCA, OHSU_TASK_BRCA) are created, which might correspond to various steps or aspects of the data processing workflow, such as filtering tasks or specific analyses related to BRCA, which is often a reference to breast cancer research.
Finally, the script writes a DataFrame out_df_filtered to a CSV file, specifying that headers should be included and that the separator should be a comma.

In [ ]:
if not OHSU_BRCA_NEW:
    PATH_DATA = create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_BRCA,NAME_TABLES])
if OHSU_BRCA_NEW:
    PATH_DATA = create_path.create_path(SAVE_DIR,[DIR_CSV,DIR_BRCA,NAME_TABLES,'OHSU_BRCA_NEW',''])
path_filtering=create_path.create_path(PATH_DATA,[NAME_FILTERING_BRCA])
path_ETH=create_path.create_path(PATH_DATA,[NAME_ETH_TASK_BRCA])
path_OHSU=create_path.create_path(PATH_DATA,[NAME_OHSU_TASK_BRCA])

out_df_filtered.to_csv(path_filtering,header=True,sep=';')